In [1]:
import pandas as pd

relacoes_entre_cnpjs = pd.read_csv(
    '/home/pedro/code/trabalho/M04/M04_2021/data/input/relacao_cnpjs.csv',
    header=None,
    dtype=str,
    sep=' '
)

informacoes_licitacoes = pd.read_csv(
    '/home/pedro/code/trabalho/M04/M04_2021/data/input/infos_licitacoes.csv',
    dtype=str,
    sep=';'
)

cnpjs_por_licitacao = pd.read_csv(
    '/home/pedro/code/trabalho/M04/M04_2021/data/input/licitacoes_cnpjs_licitantes.csv',
    dtype=str,
    sep=';'
)

# Encontrando cliques de cnpjs relacionados para municipios e licitacoes

### Vamos analisar as licitações em nível municipal

#### Começamos selecionando uma cidade:

In [2]:
cidades = informacoes_licitacoes['nom_entidade'].unique()
index_cidade_escolhida = list(cidades).index('Rio Pardo de Minas')

#### Escolhemos o município de Rio Pardo de Minas.

1. Selecionamos as licitações correspondentes ao município.

In [3]:
licitacoes_do_municipio = informacoes_licitacoes.loc[informacoes_licitacoes['nom_entidade'] == cidades[index_cidade_escolhida]]
licitacoes_do_municipio = licitacoes_do_municipio['seq_dim_licitacao']

2. Filtramos o arquivo de cnpjs licitantes pelas licitações do município.

In [4]:
cnpjs_licitantes = cnpjs_por_licitacao[cnpjs_por_licitacao['seq_dim_licitacao'].isin(licitacoes_do_municipio)]

#### 3. Criacao do grafo de CNPJs relacionados dentro do contexto municipal

In [5]:
import networkx as nx

G = nx.Graph()

# deve existir uma forma mais inteligente de fazer isso usando funcoes nativas
for cnpj in cnpjs_licitantes['num_documento']:
    relacoes_do_cnpj = relacoes_entre_cnpjs.loc[relacoes_entre_cnpjs[0] == cnpj]

    for relacao in relacoes_do_cnpj.values:
        G.add_edge(relacao[0], relacao[1])

cliques = list(nx.find_cliques(G))
tamanho_cliques = [len(clique) for clique in cliques]
max(tamanho_cliques)

78

In [6]:
nx.density(G)

0.018561209891681994

- Rio Pardo de Minas foi escolhida por conter a maior max_clique no arquivo RELATORIO_GRAFOS_POR_MUNICIPIO.csv.

- O campo max_clique contem o valor de 77, aqui o valor aparece como 78. 

- Nao tenho certeza se a max clique seria somente a clique de tamanho maximo dentre as cliques do municipio, que e o que calculamos aqui.

- O valor de densidade retornado pela funcao **density** tambem difere do valor de 0,0547031820006633 observado no relatorio.